In [1]:
# all needed imports
import pandas as pd
from process import run_ols
from sklearn.preprocessing import OneHotEncoder

In [ ]:
import pandas as pd
# generate player data
dfs = []
for i in range(10):
    csv_name = "../data/game_data/game_" + str(i + 1) + "_data.csv"
    dfs.append(pd.read_csv(csv_name))

In [ ]:
# organize data by player and store into player data
for i in range(1000):
    player_dfs = []
    for j in range(10):
        # get data from dfs[i]
        player_dfs.append(dfs[j].loc[dfs[j]["student label"] == i + 1])
    new_df = pd.concat(player_dfs)
    new_df = new_df.rename(columns={
        "game score": "score",
        "game scheduled": "game_scheduled",
        "percent training sessions attended": "percent_training_seshes",
        "overall fitness score": "fitness_score",
        "# extra strategy sessions attended": "extra_strategy_sessions_attended",
        "hours of sleep the night before game": "hours_of_sleep",
        "early bird/night owl": "early_late",
        "previous competitive sports": "previous_sport",
        "# meals on day prior to game": "meals",
        "university year": "university_year",
    })
    new_string = "student_" + str(i + 1) + ".csv"
    new_df.to_csv("../data/player_data/" + new_string)


In [ ]:
from process import run_ols
test = pd.read_csv("../data/player_data/student_1.csv")
formula = "score ~ percent_training_seshes + fitness_score + extra_strategy_sessions_attended + hours_of_sleep + meals"
result = run_ols(formula, test)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# create game_data_one_hot
for i in range(10):
    df = dfs[i]

    # one-hot-encode
    encoder = OneHotEncoder(handle_unknown='error')

    #perform one-hot encoding on all qualitative variables
    coast_df = pd.DataFrame(encoder.fit_transform(df[["coast"]]).toarray())
    coast_df.columns = encoder.get_feature_names()

    game_scheduled_df = pd.DataFrame(encoder.fit_transform(df[["game scheduled"]]).toarray())
    game_scheduled_df.columns = encoder.get_feature_names()

    sleep_schedule_df = pd.DataFrame(encoder.fit_transform(df[["early bird/night owl"]]).toarray())
    sleep_schedule_df.columns = encoder.get_feature_names()

    year_df = pd.DataFrame(encoder.fit_transform(df[["university year"]]).toarray())
    year_df.columns = encoder.get_feature_names()

    # merge all one-hot encodings back
    new_df = df.join([coast_df, game_scheduled_df, sleep_schedule_df, year_df])
    new_df = new_df.drop(columns=["coast", "game scheduled", "early bird/night owl", "university year"])

    # add to player_data_one_hot
    new_string = "game_" + str(i + 1) + "_data_one_hot.csv"
    new_df.to_csv("../data/game_data_one_hot/" + new_string)

In [ ]:
dfs_one_hot = []
for i in range(10):
    csv_name = "../data/game_data_one_hot/game_" + str(i + 1) + "_data_one_hot.csv"
    dfs_one_hot.append(pd.read_csv(csv_name))

In [ ]:
# create player_data_one_hot
for i in range(1000):
    player_dfs = []
    for j in range(10):
        # get data from dfs[i]
        player_dfs.append(dfs_one_hot[j].loc[dfs_one_hot[j]["student label"] == i + 1])
    new_df = pd.concat(player_dfs)
    new_df = new_df.rename(columns={
        "game score": "score",
        "percent training sessions attended": "percent_training_seshes",
        "overall fitness score": "fitness_score",
        "# extra strategy sessions attended": "extra_strategy_sessions_attended",
        "hours of sleep the night before game": "hours_of_sleep",
        "previous competitive sports": "previous_sport",
        "# meals on day prior to game": "meals",
        "x0_east coast": "east_coast",
        "x0_west coast": "west_coast",
        "x0_evening": "evening",
        "x0_morning": "morning",
        "x0_noon": "noon",
        "x0_early bird": "early_bird",
        "x0_night owl": "night_owl",
        "x0_frosh": "frosh",
        "x0_junior": "junior",
        "x0_senior": "senior",
        "x0_sophomore": "soph",
    })

    new_string = "student_" + str(i + 1) + "_one_hot.csv"
    new_df.to_csv("../data/player_data_one_hot/" + new_string)

In [ ]:
# run linear regression on one_hot
test = pd.read_csv("../data/player_data_one_hot/student_1_one_hot.csv")
formula = "score ~ percent_training_seshes + fitness_score + extra_strategy_sessions_attended + hours_of_sleep + meals + east_coast + west_coast + evening + morning + noon + early_bird + night_owl + frosh + junior + senior + soph"
result = run_ols(formula, test)

In [ ]:
singular_one_hot = pd.concat(dfs_one_hot)
singular_one_hot = singular_one_hot.rename(columns={
        "game score": "score",
        "game scheduled": "game_scheduled",
        "percent training sessions attended": "percent_training_seshes",
        "overall fitness score": "fitness_score",
        "# extra strategy sessions attended": "extra_strategy_sessions_attended",
        "hours of sleep the night before game": "hours_of_sleep",
        "early bird/night owl": "early_late",
        "previous competitive sports": "previous_sport",
        "# meals on day prior to game": "meals",
        "university year": "university_year",
        "x0_east coast": "east_coast",
        "x0_west coast": "west_coast",
        "x0_early bird": "early_bird",
        "x0_night owl": "night_owl",
        "x0_sophomore": "soph",
    })
singular_one_hot.to_csv("../data/singular_one_hot.csv")


In [3]:
formula = "score ~ percent_training_seshes + fitness_score + extra_strategy_sessions_attended + hours_of_sleep + meals + east_coast + west_coast + x0_evening + x0_morning + x0_noon + early_bird + night_owl + x0_frosh + x0_junior + x0_senior + soph"
result = run_ols(formula, singular_one_hot)

                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.666
Method:                 Least Squares   F-statistic:                     833.1
Date:                Wed, 15 Mar 2023   Prob (F-statistic):               0.00
Time:                        21:17:59   Log-Likelihood:                -19398.
No. Observations:                5000   AIC:                         3.882e+04
Df Residuals:                    4987   BIC:                         3.891e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

In [2]:
singular_one_hot = pd.read_csv("../data/singular_one_hot.csv")
formula = "score ~ percent_training_seshes + fitness_score + extra_strategy_sessions_attended + hours_of_sleep + meals"
result = run_ols(formula, singular_one_hot)

                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.643
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     3605.
Date:                Wed, 15 Mar 2023   Prob (F-statistic):               0.00
Time:                        20:51:01   Log-Likelihood:                -39127.
No. Observations:               10000   AIC:                         7.827e+04
Df Residuals:                    9994   BIC:                         7.831e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep